In [1]:
import numpy as np
import pandas as pd
import re

In [12]:
seen_df = pd.read_csv("data/labels.csv")
seen_df.head()

,nacsos_id,title,abstract,publication_year,0 - relevant,2 - 1.01. CO2 rise,2 - 1.02. Changes in temperature,2 - 1.03. Seasonal change,2 - 1.04. Changes in precipitation,2 - 1.05. Sea-level rise,...,5 - 4.05. Extreme cold,6 - 5.01. Pollution,"6 - 5.02. Biodiversity loss, ecosystem & microbial change",6 - 5.03. Reduced agricultural & aquaculture productivity,6 - 5.04. Reduced labour and physical capacity,representative_sample,representative_relevant,tslug,id,oa_match
0,3240,Climate Change and Health Research in the East...,"Anthropologically induced climate change, caus...",2010.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,1,climatechangeandhealthresearchintheeasternmedi...,NaN,False
1,7070,Linking health and productivity impacts to cli...,Much of the debate on global climate change ha...,2007.0,-1.0,-1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,1,linkinghealthandproductivityimpactstoclimatepo...,W2085985167,True
2,14576,Emission abatement versus development as strat...,Poorer countries are generally believed to be ...,2005.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0,1,emissionabatementversusdevelopmentasstrategies...,W3121983995,True
3,14576,Emission abatement versus development as strat...,Poorer countries are generally believed to be ...,2005.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0,1,emissionabatementversusdevelopmentasstrategies...,W2771321758,True
4,14576,Emission abatement versus development as strat...,Poorer countries are generally believed to be ...,2005.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0,1,emissionabatementversusdevelopmentasstrategies...,W2149745596,True


In [3]:
MODEL = "climatebert"
MODEL = "scincl"
y_prefix = "0 -"

pred = np.load(f"cluster-experiments/cv_results/{MODEL}__{y_prefix}__predictions.npy")
ids = np.load(f"cluster-experiments/cv_results/{MODEL}__{y_prefix}__ids.npy", allow_pickle=True)
cols = [x for x in seen_df.columns if re.match(y_prefix, x)]
cols

['0 - relevant']

In [4]:
rel_ids = np.load(f"cluster-experiments/cv_results/{MODEL}__0 -__relevant_ids.npy", allow_pickle=True)
#unseen_df = pd.read_feather("data/0_unlabelled_documents.feather")
unseen_df = pd.read_feather("../DFID-climate-health/lancet_update/2023/data/oa_docs.feather")
unseen_df = unseen_df[unseen_df["id"].isin(rel_ids)]
cname = cols[0]
pred_df = pd.DataFrame({
    "id": ids,
    cname: pred
})
unseen_df = unseen_df.merge(pred_df)

print(unseen_df.shape)
unseen_df.head()


(50534, 10)


,id,title,abstract,mag,publication_year,cited_by_count,type,doi,tslug,0 - relevant
0,W769369402,Impact of Climate Changes on Tourism on the So...,ABSTRACT.Tourism is considered as one of the m...,769369402.0,2012.0,0,journal-article,nan,impactofclimatechangesontourismonthesouthweste...,0.977093
1,W7684504,Weather Effects on Daily Traffic Accidents and...,The impact of weather conditions on traffic sa...,7684504.0,2010.0,5,None,nan,weathereffectsondailytrafficaccidentsandfatali...,0.995471
2,W767163874,"Investigation of pharmaceuticals, personal car...",Previous studies showed the presence of multip...,767163874.0,2015.0,84,journal-article,10.1016/j.scitotenv.2015.06.041,investigationofpharmaceuticalspersonalcareprod...,0.992856
3,W762813512,Climate change and human parasitic disease.,"There is increasing understanding, globally, t...",762813512.0,2014.0,3,book-chapter,10.1079/9781780642659.0095,climatechangeandhumanparasiticdisease,0.996397
4,W762745303,Emergency Department Presentations following T...,Emergency departments see an increase in cases...,762745303.0,2015.0,5,journal-article,10.1371/journal.pone.0131196,emergencydepartmentpresentationsfollowingtropi...,0.993452


In [5]:
MODEL = "climatebert"
MODEL = "scincl"
y_prefix = "2 -"
min_samples=10
pred = np.load(f"cluster-experiments/cv_results/{MODEL}__{y_prefix}__predictions.npy")
ids = np.load(f"cluster-experiments/cv_results/{MODEL}__{y_prefix}__ids.npy", allow_pickle=True)
cols = [x for x in seen_df.columns if re.match(y_prefix, x)]
cols = [x for x in cols if seen_df[x].sum()>min_samples]
pred_df = pd.DataFrame({"id": ids})
for i, cname in enumerate(cols):
    pred_df[cname]=pred[:,i]
unseen_df = unseen_df.merge(pred_df, how="left")
unseen_df

,id,title,abstract,mag,publication_year,cited_by_count,type,doi,tslug,0 - relevant,2 - 1.02. Changes in temperature,2 - 1.03. Seasonal change,2 - 1.04. Changes in precipitation,2 - 1.06. Climate change (unspecified),2 - 1.07. Other meteorological variables
0,W769369402,Impact of Climate Changes on Tourism on the So...,ABSTRACT.Tourism is considered as one of the m...,7.693694e+08,2012.0,0,journal-article,nan,impactofclimatechangesontourismonthesouthweste...,0.977093,0.996757,0.013959,0.956197,0.021212,0.987201
1,W7684504,Weather Effects on Daily Traffic Accidents and...,The impact of weather conditions on traffic sa...,7.684504e+06,2010.0,5,None,nan,weathereffectsondailytrafficaccidentsandfatali...,0.995471,0.957871,0.026408,0.996116,0.021670,0.011050
2,W767163874,"Investigation of pharmaceuticals, personal car...",Previous studies showed the presence of multip...,7.671639e+08,2015.0,84,journal-article,10.1016/j.scitotenv.2015.06.041,investigationofpharmaceuticalspersonalcareprod...,0.992856,0.019621,0.020197,0.993103,0.009617,0.495174
3,W762813512,Climate change and human parasitic disease.,"There is increasing understanding, globally, t...",7.628135e+08,2014.0,3,book-chapter,10.1079/9781780642659.0095,climatechangeandhumanparasiticdisease,0.996397,0.354813,0.017497,0.004877,0.960913,0.006316
4,W762745303,Emergency Department Presentations following T...,Emergency departments see an increase in cases...,7.627453e+08,2015.0,5,journal-article,10.1371/journal.pone.0131196,emergencydepartmentpresentationsfollowingtropi...,0.993452,0.007928,0.096003,0.006867,0.014129,0.016054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50529,W1004004788,Environment and Human Health in the Anthropoce...,"Coastal Tanzania, a region of historical and g...",1.004005e+09,2015.0,0,dissertation,nan,environmentandhumanhealthintheanthropoceneinte...,0.997004,0.981595,0.022610,0.931149,0.848387,0.012694
50530,W10020480,"A Long, Hot Summer: Repercussions of the Frenc...","Heat waves are a horrific phenomenon, killing ...",1.002048e+07,2004.0,0,journal-article,nan,alonghotsummerrepercussionsofthefrenchheatwave,0.994922,0.869407,0.009909,0.004354,0.007820,0.002153
50531,W100193498,Climate change and farmers' adaptation,The objectives of this research are (1) to des...,1.001935e+08,2011.0,0,None,nan,climatechangeandfarmersadaptation,0.992775,0.984798,0.013960,0.959774,0.031273,0.003226
50532,W100175123,Statistical methods in studies on temperature-...,Research on the Health effects of temperature ...,1.001751e+08,2011.0,2,dissertation,10.17037/pubs.00901044,statisticalmethodsinstudiesontemperaturehealth...,0.996114,0.997236,0.009631,0.006613,0.007848,0.005049


In [6]:
MODEL = "climatebert"
MODEL = "scincl"
y_prefix = "3 -"
min_samples=10
pred = np.load(f"cluster-experiments/cv_results/{MODEL}__{y_prefix}__predictions.npy")
ids = np.load(f"cluster-experiments/cv_results/{MODEL}__{y_prefix}__ids.npy", allow_pickle=True)
cols = [x for x in seen_df.columns if re.match(y_prefix, x)]
cols = [x for x in cols if seen_df[x].sum()>min_samples]
pred_df = pd.DataFrame({"id": ids})
for i, cname in enumerate(cols):
    pred_df[cname]=pred[:,i]
unseen_df = unseen_df.merge(pred_df, how="left")
unseen_df

,id,title,abstract,mag,publication_year,cited_by_count,type,doi,tslug,0 - relevant,...,2 - 1.07. Other meteorological variables,3 - 2.01. Food security,3 - 2.02. Mental health and sentiment,3 - 2.03. Cardiorespiratory disease,3 - 2.04. Direct injury and death,3 - 2.05. Infectious diseases,3 - 2.06. Other health impacts,3 - 2.07. Water security,3 - 2.08. Mortality and morbidity,3 - 2.09. Health system capacity
0,W769369402,Impact of Climate Changes on Tourism on the So...,ABSTRACT.Tourism is considered as one of the m...,7.693694e+08,2012.0,0,journal-article,nan,impactofclimatechangesontourismonthesouthweste...,0.977093,...,0.987201,0.018357,0.012384,0.003369,0.022184,0.024920,0.374989,0.022779,0.022186,0.023872
1,W7684504,Weather Effects on Daily Traffic Accidents and...,The impact of weather conditions on traffic sa...,7.684504e+06,2010.0,5,None,nan,weathereffectsondailytrafficaccidentsandfatali...,0.995471,...,0.011050,0.014615,0.007935,0.011018,0.240793,0.023636,0.070207,0.008099,0.588609,0.008392
2,W767163874,"Investigation of pharmaceuticals, personal car...",Previous studies showed the presence of multip...,7.671639e+08,2015.0,84,journal-article,10.1016/j.scitotenv.2015.06.041,investigationofpharmaceuticalspersonalcareprod...,0.992856,...,0.495174,0.183283,0.029547,0.024209,0.020452,0.167129,0.614670,0.803965,0.019909,0.035511
3,W762813512,Climate change and human parasitic disease.,"There is increasing understanding, globally, t...",7.628135e+08,2014.0,3,book-chapter,10.1079/9781780642659.0095,climatechangeandhumanparasiticdisease,0.996397,...,0.006316,0.016025,0.007463,0.006275,0.005143,0.814352,0.061612,0.023723,0.007220,0.007186
4,W762745303,Emergency Department Presentations following T...,Emergency departments see an increase in cases...,7.627453e+08,2015.0,5,journal-article,10.1371/journal.pone.0131196,emergencydepartmentpresentationsfollowingtropi...,0.993452,...,0.016054,0.039432,0.028394,0.032233,0.915038,0.034100,0.117273,0.019305,0.104885,0.060250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50529,W1004004788,Environment and Human Health in the Anthropoce...,"Coastal Tanzania, a region of historical and g...",1.004005e+09,2015.0,0,dissertation,nan,environmentandhumanhealthintheanthropoceneinte...,0.997004,...,0.012694,0.005582,0.007617,0.018601,0.004302,0.514868,0.156867,0.008579,0.010506,0.009779
50530,W10020480,"A Long, Hot Summer: Repercussions of the Frenc...","Heat waves are a horrific phenomenon, killing ...",1.002048e+07,2004.0,0,journal-article,nan,alonghotsummerrepercussionsofthefrenchheatwave,0.994922,...,0.002153,0.014349,0.011518,0.016159,0.498322,0.017058,0.020592,0.013205,0.647601,0.032812
50531,W100193498,Climate change and farmers' adaptation,The objectives of this research are (1) to des...,1.001935e+08,2011.0,0,None,nan,climatechangeandfarmersadaptation,0.992775,...,0.003226,0.761232,0.026175,0.006181,0.071827,0.154537,0.059713,0.129595,0.007975,0.016905
50532,W100175123,Statistical methods in studies on temperature-...,Research on the Health effects of temperature ...,1.001751e+08,2011.0,2,dissertation,10.17037/pubs.00901044,statisticalmethodsinstudiesontemperaturehealth...,0.996114,...,0.005049,0.015393,0.013110,0.014772,0.008437,0.013073,0.857115,0.015066,0.078998,0.017998


In [7]:
MODEL = "climatebert"
MODEL = "scincl"
y_prefix = "4 -"
min_samples=10
pred = np.load(f"cluster-experiments/cv_results/{MODEL}__{y_prefix}__predictions.npy")
ids = np.load(f"cluster-experiments/cv_results/{MODEL}__{y_prefix}__ids.npy", allow_pickle=True)
cols = [x for x in seen_df.columns if re.match(y_prefix, x)]
cols = [x for x in cols if seen_df[x].sum()>min_samples]
pred_df = pd.DataFrame({"id": ids})
for i, cname in enumerate(cols):
    pred_df[cname]=pred[:,i]

unseen_df = unseen_df.merge(pred_df, how="left")
# Exclude scenarios
include_cols = [x for x in cols if "Scenarios" not in x]
s_cols = [x for x in cols if "Scenarios" in x]
print(unseen_df.shape)
unseen_df = unseen_df[
    (unseen_df[include_cols].max(axis=1)>=0.5) #|
    #(unseen_df[include_cols].max(axis=1)>unseen_df[s_cols].max(axis=1))
]
print(unseen_df.shape)
unseen_df

(50534, 29)
(46615, 29)


,id,title,abstract,mag,publication_year,cited_by_count,type,doi,tslug,0 - relevant,...,3 - 2.05. Infectious diseases,3 - 2.06. Other health impacts,3 - 2.07. Water security,3 - 2.08. Mortality and morbidity,3 - 2.09. Health system capacity,4 - 2.01. Climate change attribution,4 - 2.02. Trend attribution,4 - 2.03. Climate sensitivity,4 - 2.04. Extreme event attribution,4 - 2.05. Scenarios
0,W769369402,Impact of Climate Changes on Tourism on the So...,ABSTRACT.Tourism is considered as one of the m...,7.693694e+08,2012.0,0,journal-article,nan,impactofclimatechangesontourismonthesouthweste...,0.977093,...,0.024920,0.374989,0.022779,0.022186,0.023872,0.091990,0.059678,0.646120,0.001134,0.034406
1,W7684504,Weather Effects on Daily Traffic Accidents and...,The impact of weather conditions on traffic sa...,7.684504e+06,2010.0,5,None,nan,weathereffectsondailytrafficaccidentsandfatali...,0.995471,...,0.023636,0.070207,0.008099,0.588609,0.008392,0.008551,0.034096,0.961629,0.002545,0.007434
2,W767163874,"Investigation of pharmaceuticals, personal car...",Previous studies showed the presence of multip...,7.671639e+08,2015.0,84,journal-article,10.1016/j.scitotenv.2015.06.041,investigationofpharmaceuticalspersonalcareprod...,0.992856,...,0.167129,0.614670,0.803965,0.019909,0.035511,0.009042,0.029859,0.963267,0.002409,0.004793
3,W762813512,Climate change and human parasitic disease.,"There is increasing understanding, globally, t...",7.628135e+08,2014.0,3,book-chapter,10.1079/9781780642659.0095,climatechangeandhumanparasiticdisease,0.996397,...,0.814352,0.061612,0.023723,0.007220,0.007186,0.950691,0.034193,0.007856,0.265657,0.067329
4,W762745303,Emergency Department Presentations following T...,Emergency departments see an increase in cases...,7.627453e+08,2015.0,5,journal-article,10.1371/journal.pone.0131196,emergencydepartmentpresentationsfollowingtropi...,0.993452,...,0.034100,0.117273,0.019305,0.104885,0.060250,0.007751,0.009231,0.110605,0.970948,0.006922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50529,W1004004788,Environment and Human Health in the Anthropoce...,"Coastal Tanzania, a region of historical and g...",1.004005e+09,2015.0,0,dissertation,nan,environmentandhumanhealthintheanthropoceneinte...,0.997004,...,0.514868,0.156867,0.008579,0.010506,0.009779,0.648042,0.062192,0.147023,0.003320,0.021833
50530,W10020480,"A Long, Hot Summer: Repercussions of the Frenc...","Heat waves are a horrific phenomenon, killing ...",1.002048e+07,2004.0,0,journal-article,nan,alonghotsummerrepercussionsofthefrenchheatwave,0.994922,...,0.017058,0.020592,0.013205,0.647601,0.032812,0.020803,0.012042,0.019915,0.993967,0.023112
50531,W100193498,Climate change and farmers' adaptation,The objectives of this research are (1) to des...,1.001935e+08,2011.0,0,None,nan,climatechangeandfarmersadaptation,0.992775,...,0.154537,0.059713,0.129595,0.007975,0.016905,0.600757,0.013773,0.009787,0.649312,0.036830
50532,W100175123,Statistical methods in studies on temperature-...,Research on the Health effects of temperature ...,1.001751e+08,2011.0,2,dissertation,10.17037/pubs.00901044,statisticalmethodsinstudiesontemperaturehealth...,0.996114,...,0.013073,0.857115,0.015066,0.078998,0.017998,0.021155,0.017683,0.634802,0.169840,0.098432


In [8]:
MODEL = "climatebert"
MODEL = "scincl"
y_prefix = "5 -"
min_samples=10
pred = np.load(f"cluster-experiments/cv_results/{MODEL}__{y_prefix}__predictions.npy")
ids = np.load(f"cluster-experiments/cv_results/{MODEL}__{y_prefix}__ids.npy", allow_pickle=True)
cols = [x for x in seen_df.columns if re.match(y_prefix, x)]
cols = [x for x in cols if seen_df[x].sum()>min_samples]
pred_df = pd.DataFrame({"id": ids})
for i, cname in enumerate(cols):
    pred_df[cname]=pred[:,i]
unseen_df = unseen_df.merge(pred_df, how="left")
unseen_df

,id,title,abstract,mag,publication_year,cited_by_count,type,doi,tslug,0 - relevant,...,4 - 2.01. Climate change attribution,4 - 2.02. Trend attribution,4 - 2.03. Climate sensitivity,4 - 2.04. Extreme event attribution,4 - 2.05. Scenarios,5 - 4.01. Floods and drought,5 - 4.02. Heatwaves,5 - 4.03. Wildfires,5 - 4.04. Other extreme events,5 - 4.05. Extreme cold
0,W769369402,Impact of Climate Changes on Tourism on the So...,ABSTRACT.Tourism is considered as one of the m...,7.693694e+08,2012.0,0,journal-article,nan,impactofclimatechangesontourismonthesouthweste...,0.977093,...,0.091990,0.059678,0.646120,0.001134,0.034406,0.008496,0.008428,0.007673,0.010125,0.007353
1,W7684504,Weather Effects on Daily Traffic Accidents and...,The impact of weather conditions on traffic sa...,7.684504e+06,2010.0,5,None,nan,weathereffectsondailytrafficaccidentsandfatali...,0.995471,...,0.008551,0.034096,0.961629,0.002545,0.007434,0.009860,0.008031,0.007291,0.009729,0.007653
2,W767163874,"Investigation of pharmaceuticals, personal car...",Previous studies showed the presence of multip...,7.671639e+08,2015.0,84,journal-article,10.1016/j.scitotenv.2015.06.041,investigationofpharmaceuticalspersonalcareprod...,0.992856,...,0.009042,0.029859,0.963267,0.002409,0.004793,0.011085,0.007413,0.007825,0.010606,0.006811
3,W762813512,Climate change and human parasitic disease.,"There is increasing understanding, globally, t...",7.628135e+08,2014.0,3,book-chapter,10.1079/9781780642659.0095,climatechangeandhumanparasiticdisease,0.996397,...,0.950691,0.034193,0.007856,0.265657,0.067329,0.023548,0.967103,0.015793,0.029687,0.044328
4,W762745303,Emergency Department Presentations following T...,Emergency departments see an increase in cases...,7.627453e+08,2015.0,5,journal-article,10.1371/journal.pone.0131196,emergencydepartmentpresentationsfollowingtropi...,0.993452,...,0.007751,0.009231,0.110605,0.970948,0.006922,0.022032,0.018744,0.029861,0.961781,0.032345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46610,W1004004788,Environment and Human Health in the Anthropoce...,"Coastal Tanzania, a region of historical and g...",1.004005e+09,2015.0,0,dissertation,nan,environmentandhumanhealthintheanthropoceneinte...,0.997004,...,0.648042,0.062192,0.147023,0.003320,0.021833,0.009039,0.009138,0.007305,0.009798,0.006499
46611,W10020480,"A Long, Hot Summer: Repercussions of the Frenc...","Heat waves are a horrific phenomenon, killing ...",1.002048e+07,2004.0,0,journal-article,nan,alonghotsummerrepercussionsofthefrenchheatwave,0.994922,...,0.020803,0.012042,0.019915,0.993967,0.023112,0.029043,0.977426,0.024910,0.035580,0.046913
46612,W100193498,Climate change and farmers' adaptation,The objectives of this research are (1) to des...,1.001935e+08,2011.0,0,None,nan,climatechangeandfarmersadaptation,0.992775,...,0.600757,0.013773,0.009787,0.649312,0.036830,0.970207,0.021430,0.024681,0.024685,0.024349
46613,W100175123,Statistical methods in studies on temperature-...,Research on the Health effects of temperature ...,1.001751e+08,2011.0,2,dissertation,10.17037/pubs.00901044,statisticalmethodsinstudiesontemperaturehealth...,0.996114,...,0.021155,0.017683,0.634802,0.169840,0.098432,0.008090,0.012216,0.007214,0.008977,0.007744


In [9]:
MODEL = "climatebert"
MODEL = "scincl"
y_prefix = "6 -"
min_samples=10
pred = np.load(f"cluster-experiments/cv_results/{MODEL}__{y_prefix}__predictions.npy")
ids = np.load(f"cluster-experiments/cv_results/{MODEL}__{y_prefix}__ids.npy", allow_pickle=True)
cols = [x for x in seen_df.columns if re.match(y_prefix, x)]
cols = [x for x in cols if seen_df[x].sum()>min_samples]
pred_df = pd.DataFrame({"id": ids})
for i, cname in enumerate(cols):
    pred_df[cname]=pred[:,i]
unseen_df = unseen_df.merge(pred_df, how="left")
unseen_df

,id,title,abstract,mag,publication_year,cited_by_count,type,doi,tslug,0 - relevant,...,4 - 2.04. Extreme event attribution,4 - 2.05. Scenarios,5 - 4.01. Floods and drought,5 - 4.02. Heatwaves,5 - 4.03. Wildfires,5 - 4.04. Other extreme events,5 - 4.05. Extreme cold,6 - 5.01. Pollution,6 - 5.03. Reduced agricultural & aquaculture productivity,6 - 5.04. Reduced labour and physical capacity
0,W769369402,Impact of Climate Changes on Tourism on the So...,ABSTRACT.Tourism is considered as one of the m...,7.693694e+08,2012.0,0,journal-article,nan,impactofclimatechangesontourismonthesouthweste...,0.977093,...,0.001134,0.034406,0.008496,0.008428,0.007673,0.010125,0.007353,0.014908,0.023415,0.031832
1,W7684504,Weather Effects on Daily Traffic Accidents and...,The impact of weather conditions on traffic sa...,7.684504e+06,2010.0,5,None,nan,weathereffectsondailytrafficaccidentsandfatali...,0.995471,...,0.002545,0.007434,0.009860,0.008031,0.007291,0.009729,0.007653,0.019361,0.013893,0.013020
2,W767163874,"Investigation of pharmaceuticals, personal car...",Previous studies showed the presence of multip...,7.671639e+08,2015.0,84,journal-article,10.1016/j.scitotenv.2015.06.041,investigationofpharmaceuticalspersonalcareprod...,0.992856,...,0.002409,0.004793,0.011085,0.007413,0.007825,0.010606,0.006811,0.180001,0.098000,0.054979
3,W762813512,Climate change and human parasitic disease.,"There is increasing understanding, globally, t...",7.628135e+08,2014.0,3,book-chapter,10.1079/9781780642659.0095,climatechangeandhumanparasiticdisease,0.996397,...,0.265657,0.067329,0.023548,0.967103,0.015793,0.029687,0.044328,0.015586,0.013536,0.015158
4,W762745303,Emergency Department Presentations following T...,Emergency departments see an increase in cases...,7.627453e+08,2015.0,5,journal-article,10.1371/journal.pone.0131196,emergencydepartmentpresentationsfollowingtropi...,0.993452,...,0.970948,0.006922,0.022032,0.018744,0.029861,0.961781,0.032345,0.015349,0.016420,0.014584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46610,W1004004788,Environment and Human Health in the Anthropoce...,"Coastal Tanzania, a region of historical and g...",1.004005e+09,2015.0,0,dissertation,nan,environmentandhumanhealthintheanthropoceneinte...,0.997004,...,0.003320,0.021833,0.009039,0.009138,0.007305,0.009798,0.006499,0.015625,0.011764,0.013544
46611,W10020480,"A Long, Hot Summer: Repercussions of the Frenc...","Heat waves are a horrific phenomenon, killing ...",1.002048e+07,2004.0,0,journal-article,nan,alonghotsummerrepercussionsofthefrenchheatwave,0.994922,...,0.993967,0.023112,0.029043,0.977426,0.024910,0.035580,0.046913,0.010683,0.013760,0.025168
46612,W100193498,Climate change and farmers' adaptation,The objectives of this research are (1) to des...,1.001935e+08,2011.0,0,None,nan,climatechangeandfarmersadaptation,0.992775,...,0.649312,0.036830,0.970207,0.021430,0.024681,0.024685,0.024349,0.025981,0.082759,0.833953
46613,W100175123,Statistical methods in studies on temperature-...,Research on the Health effects of temperature ...,1.001751e+08,2011.0,2,dissertation,10.17037/pubs.00901044,statisticalmethodsinstudiesontemperaturehealth...,0.996114,...,0.169840,0.098432,0.008090,0.012216,0.007214,0.008977,0.007744,0.018948,0.012612,0.012324


In [22]:
#unseen_df = unseen_df.drop(columns=["mag","cited_by_count","type","doi","tslug"])
seen_df = seen_df[seen_df["0 - relevant"]==1]
seen_df = seen_df[pd.notna(seen_df["id"])]
included_df = pd.concat([
    seen_df[unseen_df.columns],
    unseen_df
]).reset_index(drop=True)
included_df.to_feather("data/included_studies.feather")
included_df

,id,title,abstract,publication_year,0 - relevant,2 - 1.02. Changes in temperature,2 - 1.03. Seasonal change,2 - 1.04. Changes in precipitation,2 - 1.06. Climate change (unspecified),2 - 1.07. Other meteorological variables,...,4 - 2.04. Extreme event attribution,4 - 2.05. Scenarios,5 - 4.01. Floods and drought,5 - 4.02. Heatwaves,5 - 4.03. Wildfires,5 - 4.04. Other extreme events,5 - 4.05. Extreme cold,6 - 5.01. Pollution,6 - 5.03. Reduced agricultural & aquaculture productivity,6 - 5.04. Reduced labour and physical capacity
0,W2021823358,Projections of heat waves with high impact on ...,"Climate change will result in more intense, mo...",2014.0,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,...,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
1,W2129447349,Apparent Temperature and Cause-Specific Mortal...,"Temperature, a key climate change indicator, i...",2011.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,W2345489509,Climate Change and the Emergent Epidemic of CK...,Climate change has led to significant rise of ...,2016.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,W2093213412,Children are likely to suffer most from our fo...,BACKGROUND: The periods of fetal and child dev...,2008.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,W615562828,Autochthonous Chikungunya Transmission and Ext...,Background Extreme precipitation events are in...,2015.0,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47268,W1004004788,Environment and Human Health in the Anthropoce...,"Coastal Tanzania, a region of historical and g...",2015.0,0.997004,0.981595,0.022610,0.931149,0.848387,0.012694,...,0.003320,0.021833,0.009039,0.009138,0.007305,0.009798,0.006499,0.015625,0.011764,0.013544
47269,W10020480,"A Long, Hot Summer: Repercussions of the Frenc...","Heat waves are a horrific phenomenon, killing ...",2004.0,0.994922,0.869407,0.009909,0.004354,0.007820,0.002153,...,0.993967,0.023112,0.029043,0.977426,0.024910,0.035580,0.046913,0.010683,0.013760,0.025168
47270,W100193498,Climate change and farmers' adaptation,The objectives of this research are (1) to des...,2011.0,0.992775,0.984798,0.013960,0.959774,0.031273,0.003226,...,0.649312,0.036830,0.970207,0.021430,0.024681,0.024685,0.024349,0.025981,0.082759,0.833953
47271,W100175123,Statistical methods in studies on temperature-...,Research on the Health effects of temperature ...,2011.0,0.996114,0.997236,0.009631,0.006613,0.007848,0.005049,...,0.169840,0.098432,0.008090,0.012216,0.007214,0.008977,0.007744,0.018948,0.012612,0.012324


In [ ]:
clim = [x for x in included_df.columns if re.match("2 -",x)]
health = [x for x in included_df.columns if re.match("3 -",x)]
links = []
mat = np.zeros((len(clim),len(health)))
data = []
for i, c in enumerate(clim):
    for j, h in enumerate(health):
        n = included_df[
            (included_df[c]>=0.5) &
            (included_df[h]>=0.5)
        ].shape[0]
        mat[i,j] = n
        links.append({"source":i,"target":len(clim)+j,"n":n})
        data.append([n,c.split(".")[2],h.split(".")[2]])

df_mat = pd.DataFrame(mat)
df_mat.columns = [x.split(".")[2] for x in health]
df_mat.index = [x.split(".")[2] for x in clim]
df_mat

In [ ]:
from mpl_sankey import sankey
import matplotlib.pyplot as plt
sdf = pd.DataFrame(data, columns=["Weight", "Climate Driver", "Health impact"])
plt.figure(figsize=(7.4, 7))
sankey(sdf, cmap=plt.get_cmap('viridis'),labels_size=8)

In [ ]:
import seaborn as sns
ax = sns.heatmap(df_mat, cmap="Blues")



In [ ]:
node_df = pd.DataFrame()
node_df = pd.concat([node_df,pd.DataFrame({"name":df_mat.index,"group":0,"impact":np.arange(len(df_mat.index))})])
node_df = pd.concat([node_df,pd.DataFrame({"name":df_mat.columns,"group":1,"outcome":np.arange(len(df_mat.columns))})]).reset_index(drop=True).reset_index()
node_df

In [ ]:
link_df = pd.DataFrame.from_dict(links)
link_df

In [ ]:
import numpy as np
import holoviews as hv
from holoviews import opts, dim
hv.extension('bokeh')
hv.output(size=350)

node_hv = hv.Dataset(node_df, 'index')

chord = hv.Chord((link_df, node_hv))
chord.opts(
    opts.Chord( 
               labels = 'node', label_text_font_size='12pt', 
               node_color='node', node_cmap='Category20', node_size=10, 
               edge_color='number', edge_cmap='Category20', edge_alpha=0.9, edge_line_width=1)
)

In [ ]:
chord = hv.Chord((link_df, node_hv)).select(value=(5, None))
chord.opts(
    opts.Chord(cmap='Category10', edge_cmap='Category10', edge_color=dim('target').str(), 
               labels='name', node_color=dim('outcome').str()))